In [13]:
import os
import numpy as np
import shutil
from random import uniform
from sklearn.model_selection import train_test_split

def split(arr, ratio):
    return train_test_split(arr, test_size=ratio)

#### train/test split for Median dataset

In [20]:
samples = {
    "forearm" : {
        "HM" : {
            "str" : 641,
            "end" : 802,
            "image" : "/home/dongik/datasets/backup-median/newset/forearm_train/forearm_HM70A",
            "mask" : "/home/dongik/datasets/backup-median/newset/forearm_target/forearm_HM70A"
        },
        "SN" : {
            "str" : 1114,
            "end" : 1304,
            "image" : "/home/dongik/datasets/backup-median/newset/forearm_train/forearm_miniSONO",
            "mask" : "/home/dongik/datasets/backup-median/newset/forearm_target/forearm_miniSONO"
        }
    },
    "wrist" : {
        "HM" : {
            "str" : 0,
            "end" : 640,
            "image" : "/home/dongik/datasets/backup-median/newset/wrist_train/wrist_HM70A",
            "mask" : "/home/dongik/datasets/backup-median/newset/wrist_target/wrist_HM70A"
        },
        "SN" : {
            "str" : 803,
            "end" : 1113,
            "image" : "/home/dongik/datasets/backup-median/newset/wrist_train/wrist_miniSONO",
            "mask" : "/home/dongik/datasets/backup-median/newset/wrist_target/wrist_miniSONO"
        }
    }
}

Kfold = {
    "k" : 3,
    "dataset" : {

    }
}

assert( Kfold['k'] > 1 )

for r, v in samples.items():
    for m, lst in v.items():
        LEN = lst['end'] - lst['str'] + 1
        print(f"{r}/{m}/{LEN}")
        
        Arr = np.array([i + lst['str'] for i in range(LEN)])
        
        arr, valid = Arr, set(Arr.tolist())
        for i in range(Kfold['k'] - 1):
            arr, Kfold['dataset'][f'{i}'] = split(arr, 1/(Kfold['k'] - i))
            if i == Kfold['k'] - 2:
                Kfold['dataset'][f'{i+1}'] = arr
            valid -= set(arr.tolist())

        assert(len(valid) > 0)

        for i in range(Kfold['k']):
            vset = Kfold['dataset'][f'{i}'].tolist()
            tset = list(set(Arr.tolist()) - set(vset))

            print(f"\t({i+1}) vset: {len(vset)}, tset: {len(tset)}, Sum: {len(vset) + len(tset)}")

            if not os.path.exists(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}"):
                os.makedirs(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}")
  
            with open(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}/val.txt", 'w') as f:
                f.write('\n'.join(str(l) for l in vset))
            
            with open(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}/train.txt", 'w') as f:
                f.write('\n'.join(str(l) for l in tset))


forearm/HM/162
	(1) vset: 54, tset: 108, Sum: 162
	(2) vset: 54, tset: 108, Sum: 162
	(3) vset: 54, tset: 108, Sum: 162
forearm/SN/191
	(1) vset: 64, tset: 127, Sum: 191
	(2) vset: 64, tset: 127, Sum: 191
	(3) vset: 63, tset: 128, Sum: 191
wrist/HM/641
	(1) vset: 214, tset: 427, Sum: 641
	(2) vset: 214, tset: 427, Sum: 641
	(3) vset: 213, tset: 428, Sum: 641
wrist/SN/311
	(1) vset: 104, tset: 207, Sum: 311
	(2) vset: 104, tset: 207, Sum: 311
	(3) vset: 103, tset: 208, Sum: 311


In [18]:
prefix = "/home/dongik/datasets/median"

for r, v in samples.items():
    for m, lst in v.items():

        for fname in os.listdir(lst['image']):
            ftype = fname.split('.')[-1]
            id = int(fname.split('.')[0])

            if id < lst['str'] or id > lst['end']:
                raise Exception

            shutil.copy(
                src=os.path.join(
                    lst['image'],
                    fname
                ),
                dst=os.path.join(
                    prefix,
                    r,
                    m,
                    f"median ({id}).{ftype}"
                )
            )

        for fname in os.listdir(lst['mask']):
            ftype = fname.split('.')[-1]
            id = int(fname.split('.')[0])

            if id < lst['str'] or id > lst['end']:
                raise Exception

            shutil.copy(
                src=os.path.join(
                    lst['mask'],
                    fname
                ),
                dst=os.path.join(
                    prefix,
                    r,
                    m,
                    f"median ({id})_mask.{ftype}"
                )
            )

#### train/val split for peroneal dataset

In [51]:
prefix = "/home/dongik/datasets/_peroneal/UN/"
dst = "/home/dongik/datasets/peroneal/"

In [59]:
des = {}
for f in sorted(os.listdir(os.path.join(prefix, 'Images'))):
    format = f.split('.')[-1]
    fname = f.split('.')[0].split('_')

    r = fname[0]
    id = fname[1]

    plural = len(fname) > 2
    if plural:
        idf = f'{id}_{fname[-1]}'
    else:
        idf = f'{id}'

    if r not in des.keys():
        des[r] = {
            idf : 1
        }
    else:
        if idf not in des[r].keys():
            des[r][idf] = 1
        else:
            raise Exception
    
    if not os.path.exists(os.path.join(dst, r, 'UN')):
        os.makedirs(os.path.join(dst, r, 'UN'))
        os.makedirs(os.path.join(dst, 'splits', r, 'UN'))
    
    if len(fname) > 2:
        fn = f'peroneal ({id}_{fname[-1]}).{format}'
    else:
        fn = f'peroneal ({id}).{format}'

    shutil.copy(
        src=os.path.join(
            prefix,
            'Images',
            f
        ),
        dst=os.path.join(
            dst,
            r,
            'UN',
            fn
        )
    )

for f in sorted(os.listdir(os.path.join(prefix, 'Masks'))):
    format = f.split('.')[-1]
    fname = f.split('.')[0].split('_')

    r = fname[0]
    id = fname[1]
    
    plural = len(fname) > 3
    if plural:
        mn = f'peroneal ({id}_{fname[-2]})_mask.{format}'
    else:
        mn = f'peroneal ({id})_mask.{format}'

    shutil.copy(
        src=os.path.join(
            prefix,
            'Masks',
            f
        ),
        dst=os.path.join(
            dst,
            r,
            'UN',
            mn
        )
    )

In [60]:
Kfold = {
    "k" : 5,
    "dataset" : {

    }
}

assert( Kfold['k'] > 1 )

s = 0
for r, v in des.items():
    
    Arr = np.array(list(v.keys()))
    s += len(Arr)

    if len(Arr) / Kfold['k'] < 1.0:
        print(f'{r} is too small, {len(Arr)}')
        continue
    
    arr, valid = Arr, set(Arr.tolist())
    for i in range(Kfold['k'] - 1):
        arr, Kfold['dataset'][f'{i}'] = split(arr, 1/(Kfold['k'] - i))
        if i == Kfold['k'] - 2:
            Kfold['dataset'][f'{i+1}'] = arr
        valid -= set(arr.tolist())

    assert(len(valid) > 0)

    for i in range(Kfold['k']):
        vset = Kfold['dataset'][f'{i}'].tolist()
        tset = list(set(Arr.tolist()) - set(vset))

        #print(f"\t({i+1}) vset: {len(vset)}, tset: {len(tset)}, Sum: {len(vset) + len(tset)}")

        if not os.path.exists(f"/home/dongik/datasets/peroneal/splits/{r}/UN/v{Kfold['k']}/{i}"):
            os.makedirs(f"/home/dongik/datasets/peroneal/splits/{r}/UN/v{Kfold['k']}/{i}")

        with open(f"/home/dongik/datasets/peroneal/splits/{r}/UN/v{Kfold['k']}/{i}/val.txt", 'w') as f:
            f.write('\n'.join(str(l) for l in vset))
        
        with open(f"/home/dongik/datasets/peroneal/splits/{r}/UN/v{Kfold['k']}/{i}/train.txt", 'w') as f:
            f.write('\n'.join(str(l) for l in tset))
            
print(f'total: {s}')

FN+10 is too small, 1
FN+11 is too small, 4
FN+12 is too small, 1
FN+13 is too small, 1
FN+15 is too small, 1
FN+7 is too small, 4
FN+9 is too small, 3
FN-1 is too small, 4
POP+1 is too small, 2
POP+2 is too small, 4
POP+3 is too small, 1
POP+5 is too small, 2
total: 490


In [41]:
s = 0
for k, v in des.items():
    s += v['cnt']
    print(k, ':', v['cnt'])
    for id, cnt in v['id'].items():
        if cnt > 1:
            #print(id, ':', cnt)
            pass

print('total:', s)

FH : 91
FN+10 : 1
FN+11 : 4
FN+12 : 1
FN+13 : 1
FN+15 : 1
FN+1 : 77
FN+2 : 58
FN+3 : 49
FN+4 : 29
FN+5 : 16
FN+6 : 6
FN+7 : 4
FN+8 : 8
FN+9 : 3
FN-1 : 4
FN : 106
POP+1 : 2
POP+2 : 4
POP+3 : 1
POP+5 : 2
POP : 22
total: 490


In [ ]:
arr, valid = Arr, set(Arr.tolist())
for i in range(Kfold['k'] - 1):
    arr, Kfold['dataset'][f'{i}'] = split(arr, 1/(Kfold['k'] - i))
    if i == Kfold['k'] - 2:
        Kfold['dataset'][f'{i+1}'] = arr
    valid -= set(arr.tolist())

assert(len(valid) > 0)

for i in range(Kfold['k']):
    vset = Kfold['dataset'][f'{i}'].tolist()
    tset = list(set(Arr.tolist()) - set(vset))

    print(f"\t({i+1}) vset: {len(vset)}, tset: {len(tset)}, Sum: {len(vset) + len(tset)}")

    if not os.path.exists(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}"):
        os.makedirs(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}")

    with open(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}/val.txt", 'w') as f:
        f.write('\n'.join(str(l) for l in vset))
    
    with open(f"/home/dongik/datasets/median/splits/{r}/{m}/v{Kfold['k']}/{i}/train.txt", 'w') as f:
        f.write('\n'.join(str(l) for l in tset))

#### train/test split for BUSI dataset

In [5]:
import os
from random import uniform
from sklearn.model_selection import train_test_split
import numpy as np

"""Dataset (BUSI_with_GT) description: 
benign          (437 samples)
malignant       (210 samples)
normal          (133 samples)
splits
    benign
        v5      0/train.txt or 0/val.txt
    malignant
        v5      0/train.txt or 0/val.txt
    normal
        v5      0/train.txt or 0/val.txt
"""

def tts(idx, test_size):
    return train_test_split(idx, test_size=test_size)

L = {
    'benign' : 437,
    'malignant' : 210,
    'normal' : 133
}
K = 4

for r, LEN in L.items():
    val = []
    idx = np.array([i + 1 for i in range(LEN)])
    for k in range(K - 1):
        idx, v = tts(idx, 1 / (K - k))
        val.append(v)
    val.append(idx)

    idx = np.array([i + 1 for i in range(LEN)])
    for i in range(K):
        v = val[i].tolist()
        t = list(set(idx.tolist()) - set(v))
        print(f'{i} - v: {len(v)}, t: {len(t)}, total: {len(v) + len(t)}')
        if len(list(set(v) & set(t))) > 0:
            raise Exception
        if not os.path.exists(f'/home/dongik/datasets/BUSI_with_GT/splits/{r}/v{K}/{i}'):
            os.makedirs(f'/home/dongik/datasets/BUSI_with_GT/splits/{r}/v{K}/{i}')
        with open(f'/home/dongik/datasets/BUSI_with_GT/splits/{r}/v{K}/{i}/val.txt', 'w') as f:
            f.write('\n'.join(str(l) for l in v))
        with open(f'/home/dongik/datasets/BUSI_with_GT/splits/{r}/v{K}/{i}/train.txt', 'w') as f:
            f.write('\n'.join(str(l) for l in t))

0 - v: 110, t: 327, total: 437
1 - v: 109, t: 328, total: 437
2 - v: 109, t: 328, total: 437
3 - v: 109, t: 328, total: 437
0 - v: 53, t: 157, total: 210
1 - v: 53, t: 157, total: 210
2 - v: 52, t: 158, total: 210
3 - v: 52, t: 158, total: 210
0 - v: 34, t: 99, total: 133
1 - v: 33, t: 100, total: 133
2 - v: 33, t: 100, total: 133
3 - v: 33, t: 100, total: 133
